<a href="https://colab.research.google.com/github/Madhavi-1234/pytorch-from-scratch/blob/main/Pytorch_training_using_dataset_and_dataLoader_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import load_breast_cancer
import torch.nn as nn


In [2]:
data= load_breast_cancer()

In [3]:
df= pd.DataFrame(data.data, columns= data.feature_names)

In [4]:
df['target'] = data.target

In [5]:
x=df.drop('target', axis=1)

In [6]:
y= df['target']

In [7]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [42]:
x_train,x_test,y_train,y_test= train_test_split(x,y, test_size=0.2,random_state= 42)

In [43]:
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.fit_transform(x_test)

In [44]:
y_train

,target
68,1
181,0
63,1
248,1
60,1
...,...
71,1
106,1
270,1
435,0


In [45]:
y_train

,target
68,1
181,0
63,1
248,1
60,1
...,...
71,1
106,1
270,1
435,0


In [46]:
x_train_tensor= torch.from_numpy(x_train).float()
x_test_tensor= torch.from_numpy(x_test).float()
y_train_tensor = torch.from_numpy(y_train.values).float()
y_test_tensor= torch.from_numpy(y_test.values).float()

In [54]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):

 def __init__(self, features, labels):

    self.features= features
    self.labels= labels

 def __len__(self):
    return len(self.features)

 def __getitem__(self,index):
    return self.features[index] , self.labels[index]



In [55]:
train_dataset= CustomDataset(x_train_tensor, y_train_tensor)
test_dataset= CustomDataset(x_test_tensor, y_test_tensor)

In [56]:
train_dataset[10]

(tensor([-0.4976,  0.6137, -0.4981, -0.5310, -0.5769, -0.1749, -0.3622, -0.2849,
          0.4335,  0.1782, -0.3684,  0.5531, -0.3167, -0.4052,  0.0403, -0.0380,
         -0.1804,  0.1648, -0.1217,  0.2308, -0.5004,  0.8194, -0.4692, -0.5331,
         -0.0491, -0.0416, -0.1491,  0.0968,  0.1062,  0.4904]),
 tensor(1.))

In [57]:
train_loader= DataLoader(train_dataset, batch_size= 32, shuffle= True)
test_loader=  DataLoader(test_dataset, batch_size= 32, shuffle= True)

# Defining Model

In [58]:
import torch
import torch.nn as nn

class MySimpleNN(nn.Module):

    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, features):
        out = self.linear(features)
        out = self.sigmoid(out)
        return out

In [59]:
learning_rate= 0.1
epochs= 25

In [61]:
# create model
model= MySimpleNN(x_train_tensor.shape[1])

# define the optimizer
optimizer= torch.optim.SGD(model.parameters(), lr= learning_rate)

# define loss function
loss_fn= nn.BCELoss()

# Training Pipeline

In [62]:
# define loop
for epoch in range(epochs):

  for batch_features , batch_labels in train_loader:

     # forward pass
     y_pred= model(x_train_tensor)

     #loss calcualte
     loss= loss_fn(y_pred, y_train_tensor.view(-1,1))

     # clear gradients
     optimizer.zero_grad()

     # backward pass
     loss.backward()

     # parametrs update
     optimizer.step()

# print loass in each epoch
print(f" Epoch: { epoch+ 1}, Loss: {loss.item()}")

 Epoch: 25, Loss: 0.07488124817609787


## Evaluation


In [63]:
# model evaluation
model.eval()   # set the model to evaluate mode
accuracy_list= []

with torch.no_grad():
  for batch_features, batch_labels in test_loader:
    # forward pass
    y_pred= model(batch_features)
    y_pred= (y_pred > 0.6).float()  # convert probabilities to binary prediction

    # calculate the accuracy for current batch
    batch_accuracy=  (y_pred.view(-1) == batch_labels). float(). mean().item()
    accuracy_list.append(batch_accuracy)

# calculate overall accuracy
overall_accuracy=  sum(accuracy_list)/ len(accuracy_list)
print(f" Accuracy: { overall_accuracy: .4f}")

 Accuracy:  0.9922
